In [1]:
import os
import sys

import numpy as np
from scipy.linalg import diagsvd
import matplotlib.pyplot as plt

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from compute_energy import compute_energy
from imaginary_time_evolution import ImaginaryTimeEvolution, get_heisenberg_local_hamiltonian

# Keep 3 digits of accuracy, and avoid scientific notation when possible.
np.set_printoptions(precision=3, suppress=True, threshold=5) 

### Goal: 
1. Apply Imaginary Time Evolution Operator successively between all NN-pairs.
2. Use an SVD to bring the time-evolved state back into MPS form, with bond dimension back to ``D`` (in mixed canonical form wrt to the bond being truncated)

In [2]:
def create_random_state(n_spins):
    random_state_unnormed = np.random.uniform(low=-1., high=1., size= 2**n_spins)
    return random_state_unnormed / np.linalg.norm(random_state_unnormed) 

In [3]:
def decompose_to_mps(state, max_bond_dim=None):
    """
    @param state: is a np.ndarray of shape [2**n_sites]
    """

    n_sites = int(np.log2(state.size)) # First we try with 4

    if max_bond_dim is None: 
        # If the user doesn't specify a max_bond_dim, then any dimension is allowed
        # And we can recreate the state exactly.
        max_bond_dim = 2**(n_sites//2)

    mps = []
    prev_u_dim = 1
        
    for i in range(n_sites-1):
        state = state.reshape((2*prev_u_dim, -1)) 
        
        # FULL BOND DIMENSION [2, 4, 8, 16 ... 16, 8, 4, 2]
        # This would preserve the state in its entirety
        if (i < n_sites / 2): 
            full_bond_dim = 2 ** (i+1)
        else:
            full_bond_dim = 2 ** (n_sites - i)
                    
        # Perform the SVD
        u, s, v = np.linalg.svd(state, full_matrices=False)
        # state with shape [l, r]
        # u with shape [l, k]
        # s with shape k := min(l, r)
        # v with shape [k, r]

        # CONSTRAIN IF MAXIMUM ALLOWED DIMENSION < FULL REQUIRED DIMENSION
        # This amounts to changing k -> max_bond_dimension
        if max_bond_dim < full_bond_dim:
            # If our max_bond_dim is less than the full bond dimension
            # required to perfectly encode the state, then we need to 
            # "trim" our u, s, and v matrices
            
            u = u[:, :max_bond_dim]
            s = s[:max_bond_dim]
            v = v[:max_bond_dim, :]
            
        # RESHAPE U DEPENDING ON BOUNDARY CONDITIONS
        # Depending on whether we are at boundary sites or in the chain, 
        # we will need 2 or 3 indices, respectively.
        if (i == 0):
            u = u.reshape([2, 1, 2])
            #elif (n_sites % 2 == 1 and i == n_sites // 2): # Middle site
            #    u = u.reshape([prev_u_dim, 2, prev_u_dim])
        elif (i == n_sites - 1):
            u = u.reshape([2, 2, 1])
        else: 
            u = u.reshape([prev_u_dim, 2, -1])
              
        # The first axis on the next U has the same shape as the last axis of the current U
        prev_u_dim = u.shape[-1]

        # We add our U to the matrix product state list
        # This is a list because the U's can have different numbers of indices
        mps.append(u)
        state = np.dot(np.diag(s), v)
        
    mps.append(state)
            
    return mps

In [4]:
init_state = decompose_to_mps(create_random_state(16), 4)

In [6]:
imaginary_time_evolution = ImaginaryTimeEvolution(get_heisenberg_local_hamiltonian(kind="4-tensor"), 4)
gs = imaginary_time_evolution.find_groundstate(init_state)

ValueError: shape-mismatch for sum

In [7]:
print(gs)

NameError: name 'gs' is not defined